## Metadata exploration
This code is to explore the original metadata elements. The idea is to separate it from the code and make it nicely readable.

### Part 1: Type

In [24]:
import sys
sys.path.append('../utils')
from database import dbQuery
from dotenv import load_dotenv
from rdflib import Graph
from collections import Counter
import pandas as pd
def turple2dict(rows): # transform a query result from turple to dict
    col_names = ['identifier', 'hash', 'uri', 'turtle', 'prefix']  
    return [dict(zip(col_names, row)) for row in rows]
load_dotenv()
sql = '''
SELECT items.identifier,items.hash,items.uri,items.turtle,sources.turtle_prefix
FROM harvest.items LEFT JOIN harvest.sources ON items.source = sources.name::text;
'''
result_items = turple2dict(dbQuery(sql, hasoutput=True))

**Q1. The count of type values from the metadata**

In [ ]:
# Note that we script does not get the latest item with the same identifier, for an insight we makes it simple to choose the first one. In the code we need to extract the latest one.
count_non_turtle = 0
count_err_query = 0
identifier_list = []

def get_element_type(item):
    global count_non_turtle
    global count_err_query
    global identifier_list
    if item['turtle'] is None:
        count_non_turtle = count_non_turtle + 1
        return None
    
    if item['prefix'] is not None:
        turtle = item['prefix'] + item['turtle']
    else:
        turtle = item['turtle']

    if item['identifier'] not in identifier_list: #to avoid duplicate identifiers
        identifier_list.append(item['identifier']) 
        try:
            g = Graph()
            g.parse(data=turtle, format="turtle")

            query = '''
            prefix dct: <http://purl.org/dc/terms/>
            SELECT ?p
            WHERE {
                ?p dct:type 'journalpaper'
            }
            '''
            results = g.query(query)
            if len(results) > 0:
                return 'journalpaper'

            query = '''
            prefix dct: <http://purl.org/dc/terms/>
            SELECT ?type
            WHERE {
                ?p dct:identifier ?identifier;
                dct:type ?type
                FILTER (?type != <http://inspire.ec.europa.eu/glossary/SpatialReferenceSystem>) 
            }
            ''' 
            results = g.query(query)
            for row in results:
                return str(row['type'])
            # return [str(row['type']) for row in results]

        except Exception as e:
            # print('exception:', e)
            count_err_query = count_err_query + 1
            return None
    else:
        return None

types = []
for item in result_items:
    types.append(get_element_type(item))
counter = Counter(types)
print('Number of records with an empty turtle:', count_non_turtle)
print('Number of records with an error when querying the type:', count_err_query)

Number of records with an empty turtle: 2821
Number of records with an error when querying the type: 23


In [10]:
# display the count in a table
types_df = pd.DataFrame.from_dict(counter, orient='index', columns=['Count'])
types_df.index.name = 'Type'
types_df = types_df.sort_values('Count', ascending=False)
types_df

,Count
Type,
journalpaper,9001
None,8870
http://inspire.ec.europa.eu/metadata-codelist/ResourceType/dataset,1799
MAP,1016
document,596
http://inspire.ec.europa.eu/metadata-codelist/ResourceType/service,272
dataset,129
http://inspire.ec.europa.eu/metadata-codelist/ResourceType/series,74
Best practices and tools,58


**Q2. The case of mutiple types in a triple**

In [ ]:

identifier_list = []

def get_element_type_multiple(item):
    global identifier_list
    if item['turtle'] is None:
        return None
    
    if item['prefix'] is not None:
        turtle = item['prefix'] + item['turtle']
    else:
        turtle = item['turtle']

    if item['identifier'] not in identifier_list: #to avoid duplicate identifiers
        identifier_list.append(item['identifier']) 
        try:
            g = Graph()
            g.parse(data=turtle, format="turtle")

            query = '''
            prefix dct: <http://purl.org/dc/terms/>
            SELECT ?p
            WHERE {
                ?p dct:type 'journalpaper'
            }
            '''
            results = g.query(query)
            if len(results) > 0:
                return None

            query = '''
            prefix dct: <http://purl.org/dc/terms/>
            SELECT ?type
            WHERE {
                ?p dct:identifier ?identifier;
                dct:type ?type
                FILTER (?type != <http://inspire.ec.europa.eu/glossary/SpatialReferenceSystem>) 
            }
            ''' 
            results = g.query(query)
            if len(results) < 2: # get exactly one or get none
                return None
            return [str(row['type']) for row in results]
        except Exception as e:
            # print('exception:', e)
            return None
    else:
        return None
types = []
for item in result_items:
    t = get_element_type_multiple(item)
    if t is not None and len(set(t)) > 1: # filter out duplicated types
        types.append(t)

print('Number of records with mutiple types:', len(types))

Number of records with mutiple types: 333


In [23]:
# types

Find out that most cases are [service, a specific service type]

In [16]:
service_type = []
for t in types:
    if 'http://inspire.ec.europa.eu/metadata-codelist/ResourceType/service' in t:
        ser_spec = [i for i in t if i != 'http://inspire.ec.europa.eu/metadata-codelist/ResourceType/service']
        service_type.extend(ser_spec)
counter_s = Counter(service_type)

service_types_df = pd.DataFrame.from_dict(counter_s, orient='index', columns=['Count'])
service_types_df.index.name = 'Service Type'
service_types_df = service_types_df.sort_values('Count', ascending=False)
service_types_df

,Count
Service Type,
http://inspire.ec.europa.eu/metadata-codelist/SpatialDataServiceType/WMS,198
http://inspire.ec.europa.eu/metadata-codelist/SpatialDataServiceType/view,73
http://inspire.ec.europa.eu/metadata-codelist/SpatialDataServiceType/download,47
http://inspire.ec.europa.eu/metadata-codelist/SpatialDataServiceType/AGIS_MAPPING,21
http://inspire.ec.europa.eu/metadata-codelist/SpatialDataServiceType/discovery,6
http://inspire.ec.europa.eu/metadata-codelist/SpatialDataServiceType/CSW,1
http://inspire.ec.europa.eu/metadata-codelist/SpatialDataServiceType/other,1


Other cases?

In [22]:
other_t = []
for t in types:
    if 'http://inspire.ec.europa.eu/metadata-codelist/ResourceType/service' not in t:
        other_t.append(t)
print('Number of multiple types not a service:', len(other_t))
# other_t

Number of multiple types not a service: 61


Combinations:
- ['Publications &amp; reports', 'Scientific'] 
- ['General articles', 'Scientific'] 
- ['EU policy document', 'Policy documents']
- ['National policy document', 'Policy documents']
- ['Policy documents', 'Science-based policy advice']
- ['Policy documents', 'Policy recommendations']
- ['Education &amp; Training material', 'Vocational education &amp; training']
- ['Education &amp; Training material', 'Secondary education'],
- ['Education &amp; Training material', 'Interview'],
- ['Education &amp; Training material', 'Higher education']


Now check the journal paper type

In [28]:
identifier_list = []
count_paper_no_type = 0
count_err_query = 0
def get_element_type_journal_paper(item):

    global identifier_list
    global count_paper_no_type
    global count_err_query
    if item['turtle'] is None:
        return None
    
    if item['prefix'] is not None:
        turtle = item['prefix'] + item['turtle']
    else:
        turtle = item['turtle']

    if item['identifier'] not in identifier_list: #to avoid duplicate identifiers
        identifier_list.append(item['identifier']) 
        try:
            g = Graph()
            g.parse(data=turtle, format="turtle")

            query = '''
            prefix dct: <http://purl.org/dc/terms/>
            SELECT ?p
            WHERE {
                ?p dct:type 'journalpaper'
            }
            '''
            results = g.query(query)
            if len(results) == 0: # only consider the journal paper
                return None
            
            query = '''
            prefix dct: <http://purl.org/dc/terms/>
            SELECT ?type
            WHERE {
                ?p dct:type ?type
                FILTER (?type != <http://inspire.ec.europa.eu/glossary/SpatialReferenceSystem>) 
            }
            ''' 
            results = g.query(query)
            if len(results) == 0: # get exactly one or get none
                count_paper_no_type += 1
                return None
            for row in results:
                return str(row['type'])
            # return [str(row['type']) for row in results]

        except Exception as e:
            # print('exception:', e)
            count_err_query = count_err_query + 1
            return None
    else:
        return None
types_p= []
for item in result_items:
    p = get_element_type_journal_paper(item)
    if p is not None:
        types_p.append(t)
print('Number of paper records without another type:', count_paper_no_type)
print('Number of records with an error when querying the type:', count_err_query)
print('Number of paper records with other types:', len(types_p))

Number of paper records without another type: 0
Number of records with an error when querying the type: 23
Number of paper records with other types: 9001


The journal paper types (> 9000 records) do not give specific indications.

**Conclusion:**

- journalpaper 9001 keep it? too general?
- http://inspire.ec.europa.eu/metadata-codelist/ResourceType/dataset	1799 keep it
- MAP	1016 ???, many in dev but none in prod
- document	596 too general?
- http://inspire.ec.europa.eu/metadata-codelist/ResourceType/service	272 too general? There is also specific service indicated
- http://inspire.ec.europa.eu/metadata-codelist/ResourceType/series	74 keep it
- Best practices and tools	58 Celine suggest to remove 'tool' and what does 'best practice' mean???
- Other	remove it
- Education &amp; Training material	38 keep it, and merge 'Vocational education and training' and 'Secondary education'
- Publications &amp; reports	change to 'reports'?
- Scientific	13 remove I guess
- Interview	10 keep it
  
- nongeographicaldataset: maybe goes to dataset
- journal-article: go to journalpaper? because there are few
- EU policy document: we combine all the policy related document to policy? with national policy, 
- software 8 : keep it
- conference deliverables: for me looks too specific?
- general articles 7: not sure, too few
- image, text, videos can be removed, there are too few and these are formats
- model is too few, I think we can remove it